In [27]:
#import os
import os

#Create a P2PKH Bitcoin Address

# P2PKH Script
from bitcoin import *
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress

Create a P2PKH Script

In [55]:
SelectParams('testnet')
# Generate a random private key
s = 'cSSyMMZQkB4uLULkEAF4CypPF2meKrwfox2sXz6BEnmMLckc4Y9E'
bytes_s = s.encode()

private_key = CBitcoinSecret.from_secret_bytes(bytes_s)
# Derive the public key and Bitcoin address
public_key = private_key.pub
address = P2PKHBitcoinAddress.from_pubkey(public_key)

print("Private Key:", private_key)
print("Public Key:", public_key.hex())
print("Bitcoin Address:", address)



Private Key: 3BJHsee9XihA97ct3L6W5qqzBEDFb4GT8WrPGALyRxtCtLEKFJ4BKsoHLzqGEUjiy4Ur5Hyay6hd3nYP
Public Key: 04588fd0d036888fb36c7b22685ff797269ad7744288991becb08a9250bb5119700039d8e32ade9821b8ea92dcdffe789c33124c11ea2c9212fbf3ede0f5c49e53
Bitcoin Address: mkpoAUVfz5egpZRT6aFG1RMYQMBY6kxRhe


Private Key: cSSyMMZQkB4uLULkEAF4CypPF2meKrwfox2sXz6BEnmMLckc4Y9E

Public Key: 03b0aef05565615f51327a9e86912bb43c7f4f14b7e8c7a1451649c51f949d2a2f

Bitcoin Address: mr2gt3V9KiBmaPsJTtWxqjaHkXcX46Nrjt

Locks Funds

Spend Locked Funds

In [1]:
#!/usr/bin/env python3

# Copyright (C) 2014 The python-bitcoinlib developers
#
# This file is part of python-bitcoinlib.
#
# It is subject to the license terms in the LICENSE file found in the top-level
# directory of this distribution.
#
# No part of python-bitcoinlib, including this file, may be copied, modified,
# propagated, or distributed except according to the terms contained in the
# LICENSE file.

"""Low-level example of how to spend a standard pay-to-pubkey-hash (P2PKH) txout"""

import hashlib

from bitcoin import SelectParams
from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import CScript, OP_DUP, OP_HASH160, OP_EQUALVERIFY, OP_CHECKSIG, SignatureHash, SIGHASH_ALL
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH
from bitcoin.wallet import CBitcoinAddress, CBitcoinSecret
#from bitcoin import broadcast_tx

SelectParams('testnet')

# Create the (in)famous correct brainwallet secret key.
h = hashlib.sha256(b'cSSyMMZQkB4uLULkEAF4CypPF2meKrwfox2sXz6BEnmMLckc4Y9E').digest()
seckey = CBitcoinSecret.from_secret_bytes(h)

# Same as the txid:vout the createrawtransaction RPC call requires
#
# lx() takes *little-endian* hex and converts it to bytes; in Bitcoin
# transaction hashes are shown little-endian rather than the usual big-endian.
# There's also a corresponding x() convenience function that takes big-endian
# hex and converts it to bytes.
txid = lx('c5cbb8836612144a69f33775810a725b715140608093e902b609f28f7ff90457')
vout = 0

# Create the txin structure, which includes the outpoint. The scriptSig
# defaults to being empty.
txin = CMutableTxIn(COutPoint(txid, vout))

# We also need the scriptPubKey of the output we're spending because
# SignatureHash() replaces the transaction scriptSig's with it.
#
# Here we'll create that scriptPubKey from scratch using the pubkey that
# corresponds to the secret key we generated above.


txin_scriptPubKey = CScript([OP_DUP, OP_HASH160, Hash160(seckey), OP_EQUALVERIFY, OP_CHECKSIG])

# Create the txout. This time we create the scriptPubKey from a Bitcoin
# address.
txout = CMutableTxOut(0.001, CBitcoinAddress('mr2gt3V9KiBmaPsJTtWxqjaHkXcX46Nrjt').to_scriptPubKey())

# Create the unsigned transaction.
tx = CMutableTransaction([txin], [txout])

#BroadCast the transaction


# Calculate the signature hash for that transaction.
sighash = SignatureHash(txin_scriptPubKey, tx, 0, SIGHASH_ALL)

# Now sign it. We have to append the type of signature we want to the end, in
# this case the usual SIGHASH_ALL.
sig = seckey.sign(sighash) + bytes([SIGHASH_ALL])

# Set the scriptSig of our transaction input appropriately.
txin.scriptSig = CScript([sig, seckey.pub])

# Verify the signature worked. This calls EvalScript() and actually executes
# the opcodes in the scripts to see if everything worked out. If it doesn't an
# exception will be raised.
VerifyScript(txin.scriptSig, txin_scriptPubKey, tx, 0, (SCRIPT_VERIFY_P2SH,))

# Done! Print the transaction to standard output with the bytes-to-hex
# function.
print(b2x(tx.serialize()))

VerifyOpFailedError: EvalScript: OP_EQUALVERIFY failed

In [3]:
from bitcoin.core import b2lx, lx, COIN, COutPoint
from bitcoin.core.script import CScript, OP_DUP, OP_HASH160, OP_EQUALVERIFY, OP_CHECKSIG
from bitcoin.wallet import CBitcoinAddress, CBitcoinSecret
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH
from bitcoin.core import CMutableTransaction, CMutableTxOut, CMutableTxIn

# Private key and Bitcoin address from the previous step
private_key = CBitcoinSecret.from_secret_bytes(b'cSSyMMZQkB4uLULkEAF4CypPF2meKrwfox2sXz6BEnmMLckc4Y9E')
address = CBitcoinAddress.from_string('mr2gt3V9KiBmaPsJTtWxqjaHkXcX46Nrjt')

# Create a transaction input (UTXO)
txid = lx('c5cbb8836612144a69f33775810a725b715140608093e902b609f28f7ff90457') # Transaction ID of the UTXO you want to spend
output_index = 0 # Index of the output in the transaction
txin = CMutableTxIn(COutPoint(txid, output_index))

# Create a transaction output to the desired destination
destination_address = CBitcoinAddress('2Mz87LWo5GgbHVZhx48nxZQfCh84opc9rzn') # Recipient’s address
amount_to_send = 0.001 # Amount to send in satoshis
txout = CMutableTxOut(amount_to_send * COIN, destination_address.to_scriptPubKey())

# Create the transaction
tx = CMutableTransaction([txin], [txout])

# Sign the transaction
sighash = SignatureHash(CScript([OP_DUP, OP_HASH160, address, OP_EQUALVERIFY, OP_CHECKSIG]), tx, 0, SIGHASH_ALL)
sig = private_key.sign(sighash) + bytes([SIGHASH_ALL])
txin.scriptSig = CScript([sig, private_key.pub])

# Verify the signature
VerifyScript(txin.scriptSig, txout.scriptPubKey, tx, 0, (SCRIPT_VERIFY_P2SH,))

# Broadcast the transaction
# This part is network-dependent and is not covered by python-bitcoinlib

AttributeError: type object 'CBitcoinAddress' has no attribute 'from_string'